# Imports

In [16]:
import polars as pl
import numpy as np
import folium
import utils

# Read Data
For this analysis, the California Department of Mine Reclemation (DMR) mine data was used to find potential new customers for a local geology consultant. DMR annually collects data from operators all over the state, and this is then used for their maps and analysis. It is available for public use via the Cal Open Data Portal in shapefile format.

In [17]:
mines_csv = pl.read_csv(r"DMR_All_Mines.csv",ignore_errors=True)

# Filter/Plot Data
All we really need is the name, location, status (ie active/closed), and who owns/operates the mine (these columns can be found in the utils.py). After selecting only those columns, we'll filter the mines data by those that are:
- active
- open pit (as this is my associates' specialty)
- have their primary product as gypsum.

In [18]:
mines_csv = mines_csv[utils.get_cols_to_keep()]

active_open_pit_gyp_mines = mines_csv.filter(
    (
    pl.col("MineStatus").str.contains("ACTIVE"))
    & (pl.col("Opt_Type").str.contains("OPEN PIT"))
    & (~pl.col("PriProduct").str.contains("NOT REPORT")
    & (pl.col("PriProduct").str.contains("Gyp"))
    )
)
active_open_pit_gyp_mines

Mine_ID,MineName,Latitude,Longitude,ReportYear,Acres_Dist,MineStatus,Operator,Opt_City,Owner,PriProduct,Opt_Type,FaceAmount
str,str,f64,f64,i64,f64,str,str,str,str,str,str,f64
"""91-13-0005""","""PLASTER CITY Q…",33.023056,-116.081944,2021,518.6,"""ACTIVE""","""UNITED STATES …","""IMPERIAL""","""UNITED STATES …","""Gypsum""","""OPEN PIT""",4.896461e6
"""91-33-0076""","""STANDARD GYPSU…",33.843056,-114.8575,2021,132.0,"""ACTIVE""","""Double D Minin…","""Cerritos""","""DOUBLE D MININ…","""Gypsum""","""OPEN PIT""",587758.0
"""91-56-0016""","""CUYAMA GYPSUM …",34.846667,-119.382222,2022,212.0,"""ACTIVE""","""H. LIMA COMPAN…","""MANTECA""","""H. LIMA COMPAN…","""Gypsum""","""OPEN PIT,QUARR…",809136.0
"""91-15-0017""","""LOST HILLS MIN…",35.655,-119.773056,2022,708.0,"""ACTIVE""","""H.M. HOLLOWAY""","""Bakersfield ""","""H.M. HOLLOWAY""","""Gypsum""","""OPEN PIT""",1.1266e6
"""91-24-0018""","""JOHN MANCEBO M…",36.918056,-120.843056,2022,0.0,"""ACTIVE""","""JOHN MANCEBO""","""DOS PALOS""","""MANCEBO""","""Gypsum""","""OPEN PIT""",5023.5


This leaves us with only 5 mines in CA. One caveat is that the consultant would rather not travel over 60 miles to a given mine from his home city of Bakersfield, so we'll draw a radius around the city to visually fitler those mines out (the consultant asked me not to filter them out of the data entiretly so they could do further reseach in case closer mines maybe don't pan out).

In [19]:
gypsum_map = folium.Map([35, -119], zoom_start=8)
distance_to_travel = (60 / 0.621371) * 1000
folium.Circle(
        location = [35.3733, -119.0187],
        radius = distance_to_travel,
    ).add_to(gypsum_map)
for name, mine in active_open_pit_gyp_mines.groupby("Mine_ID"):
    folium.Marker(
        location = [mine["Latitude"].to_list()[0],mine["Longitude"].to_list()[0]],
        tooltip=mine["Operator"].to_list()[0],
    ).add_to(gypsum_map)

display(gypsum_map)

As we can see, only two mines fit the bill: one owned by H.M. Holloway, and the other owned by H. Lima Company. The consultant already contracts with Holloway, and Lima doesn't exactly need his services as gypsum is easily found at their location. My initial thought is to open the analysis up to other types of mines in the area.

In [20]:
active_open_pit_mines = mines_csv.filter(
    (
    pl.col("MineStatus").str.contains("ACTIVE"))
    & (pl.col("Opt_Type").str.contains("OPEN PIT"))
    & (~pl.col("PriProduct").str.contains("NOT REPORT")
    )
)
print(f"There are {active_open_pit_mines.height} many active open pit mines in CA")

There are 454 many active open pit mines in CA


As we can see, theres roughly 450 more mines in CA that are active and use open pit. The consultant asked for these to be plotted in their own map, so they can perhaps do their research and see which could use his services.

In [21]:
open_pit_map = folium.Map([35, -119], zoom_start=8)
folium.Circle(
        location = [35.3733, -119.0187],
        radius = distance_to_travel,
    ).add_to(open_pit_map)
for name, mine in active_open_pit_mines.groupby("Mine_ID"):
    folium.Marker(
        location = [mine["Latitude"].to_list()[0],mine["Longitude"].to_list()[0]],
        tooltip=mine["PriProduct"].to_list()[0],
        popup=mine["Operator"].to_list()[0]
    ).add_to(open_pit_map)

display(open_pit_map)